In [1]:
import tensorflow as tf
from tensorflow.python.framework.ops import disable_eager_execution
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

#disable_eager_execution()

In [2]:
class MyModel(keras.Model):
    def __init__(self, **kwargs):
        super(MyModel, self).__init__(**kwargs)
        self.conv = layers.Conv1D(32, 3, strides=1, padding='same', activation='relu')
        self.rnn = layers.Bidirectional(layers.GRU(32, return_sequences=True))
        self.rnn = layers.GRU(32, return_sequences=True)
        self.dense = layers.Dense(5, name='signal_mask')
        
    def call(self, inputs):        
        seq_lengths = inputs[1]
        sequences = inputs[0]
        x = self.conv(sequences)
        mask = tf.sequence_mask(seq_lengths)
        x = self.rnn(x, mask=mask)
        y = self.dense(x)
        
        return y

In [3]:
batch_size = 1
time_step = 100
signal_length = 100
sequences = np.random.rand(batch_size, time_step, 1)
seq_lengths = np.array([signal_length]*batch_size)
signal_mask = np.random.randint(5, size=(batch_size, time_step))

inputs = tf.data.Dataset.from_tensor_slices((sequences, seq_lengths))
signal_masks = tf.data.Dataset.from_tensor_slices(signal_mask)
dataset = tf.data.Dataset.zip((inputs, signal_masks))
dataset = dataset.repeat(100).batch(1)

In [4]:
mymodel = MyModel()
optimizer = keras.optimizers.Adam(learning_rate=1e-3)
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

epochs = 1

for epoch in range(epochs):
    print("Start of epoch %d" % (epoch,))
    for step, (x_batch_train, y_batch_train) in enumerate(dataset):
        with tf.GradientTape() as tape:
            logits = mymodel(x_batch_train, training=True)
            loss_value = loss(y_batch_train, logits)
        grads = tape.gradient(loss_value, mymodel.trainable_weights)
        optimizer.apply_gradients(zip(grads, mymodel.trainable_weights))
        if step % 10 == 0:
            print(
                "Training loss (for one batch) at step %d: %.4f"
                % (step, float(loss_value))
            )
            print("Seen so far: %s samples" % ((step + 1) * 64))

Start of epoch 0

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Training loss (for one batch) at step 0: 1.6106
Seen so far: 64 samples
Training loss (for one batch) at step 10: 1.5911
Seen so far: 704 samples
Training loss (for one batch) at step 20: 1.5880
Seen so far: 1344 samples
Training loss (for one batch) at step 30: 1.5828
Seen so far: 1984 samples
Training loss (for one batch) at step 40: 1.5776
Seen so far: 2624 samples
Training loss (for one batch) at step 50: 1.5707
Seen so far: 3264 samples
Training loss (for one batch) at step 60: 1.5

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(((sequences, seq_lengths),  signal_mask)).repeat(10).batch(2)
batch, _ = next(iter(dataset))

In [ ]:
x = batch[0]
x

In [ ]:
conv = layers.Conv1D(32, 3, strides=1, padding='same', activation='relu')
#masked = layers.Masking(mask_value=0)

In [ ]:
x1 = conv(x)
x1

In [ ]:
x2 = masked(x1)
x2._keras_mask

In [ ]:
def model2(mask_value):
    mirrored_strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()
                        
    with mirrored_strategy.scope():
        seq_data = keras.Input(shape=(None, 1),  name="seq_data")
        seq_lengths = keras.Input(shape=(), name="seq_lengths", dtype=tf.int32)

        #mask = keras.layers.Lambda(lambda x: tf.sequence_mask(x))(seq_lengths)
        conv = layers.Conv1D(32, 3, strides=1, padding='same', activation='relu')(seq_data)
        #rnn = layers.Bidirectional(layers.GRU(32, return_sequences=True))(conv,mask=mask)
        masked = layers.Masking(mask_value=mask_value)(conv)
        rnn = layers.Bidirectional(layers.GRU(32, return_sequences=True))(masked)
        dense = layers.Dense(5, name="signal_mask")(rnn)
        model = keras.Model(inputs=[seq_data, seq_lengths], outputs=[dense])

        model.compile(optimizer=tf.keras.optimizers.Adam(
            learning_rate=0.001),
            loss=tf.keras.backend.sparse_categorical_crossentropy
        )
    
    return model

model2 = model2(0.0)
print(model2.summary())
dot_img_file = 'model_2.png'
tf.keras.utils.plot_model(model2, to_file=dot_img_file, show_shapes=True)

In [ ]:
batch_size = 1
time_step = 1000
signal_length = 1000
sequences = np.random.rand(batch_size, time_step, 1)
seq_lengths = np.array([signal_length]*batch_size)
signal_mask = np.random.randint(5, size=(batch_size, time_step))

    
dataset = tf.data.Dataset.from_tensor_slices(((sequences, seq_lengths),  signal_mask)).repeat(10).batch(1)


model2.fit(dataset, epochs=1)

In [ ]:
batch_size = 1
time_step = 6
signal_length = 6
sequences = np.random.rand(batch_size, time_step, 1)
seq_lengths = np.array([signal_length]*batch_size)
signal_mask = np.random.randint(5, size=(batch_size, time_step))

    
dataset = tf.data.Dataset.from_tensor_slices(((sequences, seq_lengths),  signal_mask)).repeat(10).batch(2)
batch = next(iter(dataset))
batch

In [ ]:
%%writefile config.yaml
trainingInput:
  scaleTier: CUSTOM
  # Configure a master worker with 1 with K80 GPUs
  masterType: n1-highcpu-16
  masterConfig:
    acceleratorConfig:
      count: 2
      type: NVIDIA_TESLA_K80
  # Configure 1 worker(s), each with 2 K80
  workerCount: 1
  workerType: n1-highcpu-16
  workerConfig:
    acceleratorConfig:
      count: 2
      type: NVIDIA_TESLA_K80

In [ ]:
%%bash

now=$(date +"%Y%m%d_%H%M%S")
JOB_NAME="multi_cpu_fashion_minst_$now"

gcloud ai-platform jobs submit training $JOB_NAME \
  --staging-bucket=gs://mlops-dev-workspace \
  --package-path=sample \
  --module-name=sample.model \
  --runtime-version=2.1 \
  --python-version=3.7 \
  --region=us-west1 \
  --config config.yaml
  

In [ ]:
!gcloud ai-platform jobs stream-logs multi_cpu_fashion_minst_20200805_040150